**Changes Done**

1) Dropped the "Item_Identifier" categorical column from the data.

2) knn regression model is used

In [1]:
# Importing libraries 

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

# Reading the datasets

In [2]:
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")

In [3]:
train.shape, test.shape

((8523, 12), (5681, 11))

# Data Preprocessing

In [4]:
train['Item_Fat_Content'] = train['Item_Fat_Content'].replace(["Low Fat", "low fat"], "LF")
train['Item_Fat_Content'] = train['Item_Fat_Content'].replace("Regular", "reg")

test['Item_Fat_Content'] = test['Item_Fat_Content'].replace(["Low Fat", "low fat"], "LF")
test['Item_Fat_Content'] = test['Item_Fat_Content'].replace("Regular", "reg")

**Replacing null values in "Item_Weight" column with the average weight of respective "Item_Type"** 

In [5]:
grouped = train.groupby("Item_Type")["Item_Weight"]

fill_values = {}
for name, group in grouped:
    fill_values[name] = group.mean()
    
fill_func = lambda g:g.fillna(fill_values[g.name])
train["Item_Weight"] = train.groupby("Item_Type")["Item_Weight"].apply(fill_func)
test["Item_Weight"] = test.groupby("Item_Type")["Item_Weight"].apply(fill_func)

In [6]:
print(train.Item_Weight.isna().sum())
print(test.Item_Weight.isna().sum())

0
0


**Create another category for missing values in "Outlet_Size" column**

In [7]:
train['Outlet_Size'] = train['Outlet_Size'].replace(np.NaN, "Unknown")
test['Outlet_Size'] = test['Outlet_Size'].replace(np.NaN, "Unknown")

In [8]:
train.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,LF,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,reg,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,LF,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,reg,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Unknown,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,LF,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [9]:
train.Item_Identifier.nunique()

1559

**Concatenating the train and test data**

In [10]:
labels = train.iloc[:, -1]
train = train.iloc[:, :-1]

In [11]:
data = pd.concat([train, test], ignore_index=True)
data.drop('Item_Identifier', axis=1, inplace=True)

In [12]:
data.shape

(14204, 10)

In [13]:
data.head()

,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,9.30,LF,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,5.92,reg,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2
2,17.50,LF,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1
3,19.20,reg,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,Unknown,Tier 3,Grocery Store
4,8.93,LF,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1


**Applying Categorical Encoding**

In [14]:
categorical_var = np.where(data.dtypes != np.float)[0]

columns_names_encod = data.columns[np.array(categorical_var)]
data_transformed = pd.get_dummies(data, columns=columns_names_encod, drop_first=True)

In [15]:
data_transformed.shape

(14204, 44)

**Splitting the data back**

In [16]:
final_train = data_transformed[:train.shape[0]]
final_test = data_transformed[train.shape[0]:]

In [17]:
final_train.shape, final_test.shape

((8523, 44), (5681, 44))

**Creating validation set from training set**

In [18]:
from sklearn.model_selection import train_test_split

X_train, X_cv, y_train, y_cv = train_test_split(final_train, labels, test_size=0.2, random_state=21, shuffle=False)

In [19]:
final_train.head()

,Item_Weight,Item_Visibility,Item_MRP,Item_Fat_Content_reg,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,...,Outlet_Establishment_Year_2007,Outlet_Establishment_Year_2009,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Size_Unknown,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,9.30,0.016047,249.8092,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,0
1,5.92,0.019278,48.2692,1,0,0,0,0,0,0,...,0,1,1,0,0,0,1,0,1,0
2,17.50,0.016760,141.6180,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
3,19.20,0.000000,182.0950,1,0,0,0,0,0,1,...,0,0,0,0,1,0,1,0,0,0
4,8.93,0.000000,53.8614,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0


In [20]:
X_train.head()

,Item_Weight,Item_Visibility,Item_MRP,Item_Fat_Content_reg,Item_Type_Breads,Item_Type_Breakfast,Item_Type_Canned,Item_Type_Dairy,Item_Type_Frozen Foods,Item_Type_Fruits and Vegetables,...,Outlet_Establishment_Year_2007,Outlet_Establishment_Year_2009,Outlet_Size_Medium,Outlet_Size_Small,Outlet_Size_Unknown,Outlet_Location_Type_Tier 2,Outlet_Location_Type_Tier 3,Outlet_Type_Supermarket Type1,Outlet_Type_Supermarket Type2,Outlet_Type_Supermarket Type3
0,9.30,0.016047,249.8092,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,1,0,0
1,5.92,0.019278,48.2692,1,0,0,0,0,0,0,...,0,1,1,0,0,0,1,0,1,0
2,17.50,0.016760,141.6180,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,1,0,0
3,19.20,0.000000,182.0950,1,0,0,0,0,0,1,...,0,0,0,0,1,0,1,0,0,0
4,8.93,0.000000,53.8614,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,0,0


In [21]:
X_train.shape, X_cv.shape

((6818, 44), (1705, 44))

**Feature Scaling**

In [22]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

scaled_train_columns = sc.fit_transform(X_train.iloc[:, [0, 1, 2]].values)
scaled_cv_columns = sc.transform(X_cv.iloc[:, [0, 1, 2]].values)
scaled_test_columns = sc.transform(final_test.iloc[:, [0, 1, 2]].values)

In [23]:
X_train.loc[:, ['Item_Weight', 'Item_Visibility', 'Item_MRP']] = scaled_train_columns
X_cv.loc[:, ['Item_Weight', 'Item_Visibility', 'Item_MRP']] = scaled_cv_columns
final_test.loc[:, ['Item_Weight', 'Item_Visibility', 'Item_MRP']] =scaled_test_columns

c:\users\amit sahu\appdata\local\programs\python\python37\lib\site-packages\pandas\core\indexing.py:494: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


# Training the data and getting results for cross-validation set

In [28]:
from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
knn_reg = KNeighborsRegressor()

param_grid = {'n_neighbors':[3,5,7,9,11],
              'weights': ['uniform', 'distance'],
              'metric': ['euclidean', 'manhattan'],
              'leaf_size':[1,3,5],
              'algorithm':['auto', 'kd_tree'],
              'n_jobs':[-1]}

#Fit the model
grid_search = GridSearchCV(knn_reg, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(X_train,y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=KNeighborsRegressor(algorithm='auto', leaf_size=30,
                                           metric='minkowski',
                                           metric_params=None, n_jobs=None,
                                           n_neighbors=5, p=2,
                                           weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'algorithm': ['auto', 'kd_tree'],
                         'leaf_size': [1, 3, 5],
                         'metric': ['euclidean', 'manhattan'], 'n_jobs': [-1],
                         'n_neighbors': [3, 5, 7, 9, 11],
                         'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
             scoring='neg_mean_squared_error', verbose=0)

In [30]:
final_model = grid_search.best_estimator_

In [31]:
grid_search.best_params_

{'algorithm': 'auto',
 'leaf_size': 1,
 'metric': 'euclidean',
 'n_jobs': -1,
 'n_neighbors': 11,
 'weights': 'uniform'}

In [32]:
y_cv_pred = final_model.predict(X_cv)

In [33]:
from sklearn.metrics import mean_squared_error
cv_rmse = np.sqrt(mean_squared_error(y_cv, y_cv_pred))


In [34]:
cv_rmse

1128.9332072119744

# Testing the model on Test Dataset

In [35]:
predictions_test = final_model.predict(final_test)

In [36]:
predictions_test = pd.Series(predictions_test)

In [37]:
frame = {'Item_Identifier': test.Item_Identifier, 'Outlet_Identifier':test.Outlet_Identifier, 
         'Item_Outlet_Sales': predictions_test}
submission = pd.DataFrame(frame)
submission.to_csv('kNN_submission_3.csv', index=False)